<a href="https://colab.research.google.com/github/DanielaNavaGarcia/Simulacion-I/blob/main/Ejercicio_5_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

# Entradas del problema
demand_vals = np.array([35,36,37,38,39,40,41,42,43,
                        44,45,46,47,48,49,50,51,52,
                        53,54,55,56,57,58,59,60])
demand_prob = np.array([0.01,0.015,0.02,0.02,0.022,0.023,0.025,0.027,0.028,
                        0.029,0.035,0.045,0.06,0.065,0.07,0.08,0.075,0.07,
                        0.065,0.06,0.05,0.04,0.03,0.016,0.015,0.005])
lead_time_vals = np.array([1,2,3])
lead_time_prob = np.array([0.30,0.40,0.30])
seasonal = {1:1.20,2:1.00,3:0.90,4:0.80,5:0.80,6:0.70,
            7:0.80,8:0.90,9:1.00,10:1.20,11:1.30,12:1.40}
# Costos
co, c_h, c_s = 100, 20/12, 50  # ordenar, inventario mensual (20 anual), faltante

# Política (r, Q) fija: aquí puedes ajustar
r = 40
Q = 50

def sim_month(months=12, reps=10000):
    costs = []
    for _ in range(reps):
        inv = r + Q  # iniciar con nivel de reorden
        pending = []  # lista de pedidos en tránsito (cantidad, meses_restantes)
        total_holding = total_short = total_order = 0.0
        for m in range(1, months+1):
            # generar demanda estocástica ajustada por estacionalidad
            d = np.random.choice(demand_vals, p=demand_prob) * seasonal[m]
            # recibir pedidos según lead time
            for i,(q,t) in enumerate(pending[:]):
                pending[i] = (q, t-1)
            rec = sum(q for q,t in pending if t<=0)
            pending = [(q,t) for (q,t) in pending if t>0]
            inv += rec
            # atender demanda
            sold = min(inv, d)
            short = max(0, d - inv)
            inv -= sold
            # ordenar si tocó
            if inv <= r:
                pending.append((Q, np.random.choice(lead_time_vals, p=lead_time_prob)))
                total_order += co
            # costos de inventario y faltantes
            total_holding += inv * c_h
            total_short += short * c_s
        costs.append(total_order + total_holding + total_short)
    return np.mean(costs), np.mean(total_holding), np.mean(total_short)

if __name__ == "__main__":
    avg_cost, avg_hold, avg_short = sim_month()
    print(f"Coste promedio anual: {avg_cost:.2f}")
    print(f"Costo inventario promedio: {avg_hold:.2f}")
    print(f"Costo faltante promedio: {avg_short:.2f}")

Coste promedio anual: 7690.21
Costo inventario promedio: 490.33
Costo faltante promedio: 6580.00
